In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# magic command for matplotlib to enable printing directly in the jupyter notebook
%matplotlib inline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
#from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split


In [2]:
# PATHS

# get working directory
path_working = Path.cwd()

path_data = str(path_working) + "\\" + "data" + "\\"

#print(path_data)


In [3]:
def create_dataframe (path):
    # Get the files from the path provided in the OP
    files = Path(path).glob('*.csv')  # .rglob to create iterator for subdirectories
    
    #Find all csv files in directory and concatenate them
    dfs = list()
    names = list()
    for file in files:
        data = pd.read_csv(file)
        # .stem is method for pathlib objects to get the filename without the extension
        # create another column in dataframe for filename
        data['file'] = file.stem
        names.append(file.stem)
        dfs.append(data)
        df = pd.concat(dfs, ignore_index=True)
    
    return df

In [4]:
df_raw = create_dataframe(path_data)

In [5]:
df_raw

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       relay1_log  relay2_log  relay3_log  relay4_log  snort_log1  snort_log2  \
0               0           0           0           0           0           0   
1               0           0           0           0           0           0   
2               0           0           0           0           0           0   
3               0           0           0           0           0           0   
4               0           0           0           0           0           0   
...           ...         ...         ...         ...         ...         ...   
78372           0           0           0           0           0           0   
78373           0           0           0           0           0           0   
78374           0           0           0           0           0           0   
78375           0           0           0           0           0           0   
78376           0           0           0           0           0           0   

       snort_log3  snort_log4    marker   file  
0               0           0  NoEvents  data1  
1               0           0  NoEvents  data1  
2               0           0  NoEvents  data1  
3               0           0  NoEvents  data1  
4               0           0  NoEvents  data1  
...           ...         ...       ...    ...  
78372           0           0   Natural  data9  
78373           0           0   Natural  data9  
78374           0           0   Natural  data9  
78375           0           0   Natural  data9  
78376           0           0   Natural  data9  

[78377 rows x 130 columns]

In [ ]:
#drop last column that shows csv filename source:
del df_raw[df_raw.columns[-1]]

df_raw

In [ ]:
print(df_raw.shape)

df_raw['marker'].value_counts()

### ---> unbalanced classes

In [ ]:
df_raw.info()

In [ ]:
# Look at datatypes of coloumns.
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )
        
show_dtypes(df_raw)

In [ ]:
# .copy() is very important otherwise its just a pointer
df = df_raw.copy()

In [ ]:
def marker_to_numeric(x):
        if x=='NoEvents': return 0
        if x=='Natural':   return 1
        if x=='Attack': return 2

df['marker'] = df['marker'].apply(marker_to_numeric)
df

In [ ]:
# Check for infinite float entries
# "np.finfo(np.float64).max" returns the maximum finite value that can be represented by a 64-bit floating point number
# "df.values >= np.finfo(np.float64).max" returns a bollean array where "np.finfo(np.float64).max" is checked
# "np.where(df.values >= np.finfo(np.float64).max)" returns a array of indices where "df.values >= np.finfo(np.float64).max" is true

np.where(df.values >= np.finfo(np.float64).max)

### ----> we have infinite float entries

In [6]:
# Check for NaN values
np.isnan(df.values.any())

NameError: name 'df' is not defined

### ----> we have no NaN values in the dataframe

In [ ]:
# Replace infinite updated data with nan

# "[np.inf, -np.inf]" specifies the values to be replaced
# "np.nan" specifies the value to replace the infinite values with.
# "inplace=True" modifies the Dataframe in place, so that the original df is updated the new values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

### ----> Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful due to the fact that there are still 72000 rows

In [ ]:
y = df['marker']

X = df.drop(['marker'],axis =1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
def print_score(m):
    print([m.score(X_train, y_train), m.score(X_valid, y_valid)])
    plot_confusion_matrix(m, X_train, y_train)
    plot_confusion_matrix(m, X_valid, y_valid)

In [ ]:
m = RandomForestClassifier(n_estimators=10, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
# UPSAMPLING:

from sklearn.utils import resample,shuffle

In [ ]:
df_minority_NoEvents = df[df['marker'] == 0]
df_minority_Natural = df[df['marker'] == 1]
df_not_minority = df[df['marker'] == 2] 

In [ ]:
#upsample the minority classes
df_minority_NoEvents_upsampled = resample(df_minority_NoEvents,random_state=42,n_samples=55663,replace=True)
df_minority_Natural_upsampled = resample(df_minority_Natural,random_state=42,n_samples=55663,replace=True)

In [ ]:
print(df_minority_NoEvents_upsampled.shape)
print(df_minority_Natural_upsampled.shape)

In [ ]:
#concatenate the upsampled dataframes
df_upsampled = pd.concat([df_minority_NoEvents_upsampled, df_minority_Natural_upsampled, df_not_minority ])
#df_upsampled

In [ ]:
y = df_upsampled['marker']

X = df_upsampled.drop(['marker'],axis =1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
#RF with Upsampled Data
m = RandomForestClassifier(n_estimators=10, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)